In [3]:
#(optional) install the ultralytics package
!pip install ultralytics

Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/1.0 MB ? eta -:--:--
   ---------------------------------------- 1.0/1.0 MB 15.2 MB/s eta 0:00:00
   ---------------------------------------- 0.0/8.1 MB ? eta -:--:--
   ---------------------- ----------------- 4.5/8.1 MB 21.6 MB/s eta 0:00:01
   ---------------------------------------- 8.1/8.1 MB 22.2 MB/s eta 0:00:00
   ---------------------------------------- 0.0/41.0 MB ? eta -:--:--
   ---- ----------------------------------- 4.7/41.0 MB 24.1 MB/s eta 0:00:02
   --------- ------------------------------ 10.0/41.0 MB 24.6 MB/s eta 0:00:02
   -------------- ------------------------- 14.7/41.0 MB 24.4 MB/s eta 0:00:02
   ------------------ --------------------- 19.1/41.0 MB 23.8 MB/s eta 0:00:01
   ----------------------- ---------------- 24.4/41.0 MB 24.0 MB/s eta 0:00:01
   --------------------------- ------------ 28.6/41.0 MB 23.7 MB/s eta 0:00:01
   -------


[notice] A new release of pip is available: 25.0.1 -> 25.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
import cv2
import numpy as np


#this section is used to draw the GUI on the bottom left of the screen
def draw_overlay(frame, current_room, current_action,previous_action_overlay):


    overlay_width, overlay_height = int(600), int(400)  # Scale by 2.5

    overlay = np.ones((overlay_height, overlay_width, 3), dtype=np.uint8) * 255  # White background


    cv2.rectangle(overlay, (0, 0), (overlay_width, overlay_height), (0, 0, 0), 4)  # Thicker black border

    # Define room coordinates
    rooms = {
        "Kitchen": (100, 200, 100, 160),
        "Bathroom": (100, 100, 100, 100),
        "Bedroom": (200, 100, 200, 100),
        "Living Room": (400, 100, 100, 260),
    }

    actions = {
        "Fridge": (101, 270, 25, 25),
        "Laying on Bed": (270, 101, 60, 60),
        "Sink": (169, 130, 30, 40),
        "Couch": (469, 200, 30, 80),
        "Drawer": (245, 101, 25, 25),
        "Dresser": (201, 101, 20, 50)
    
        #"Cupboard": (120, 339, 60, 20),
        #"Table": (160, 235, 40, 40),
    }


    # Draw rooms
    for room_name, (x, y, w, h) in rooms.items():
        color = (154, 205, 50)  # Medium sea green
        if room_name == current_room:
            color = (0, 165, 255)  # Orange highlight
        cv2.rectangle(overlay, (x, y), (x + w, y + h), color, -1)  # Filled
        cv2.rectangle(overlay, (x, y), (x + w, y + h), (255, 255, 255), 1)  # Black border
        cv2.putText(overlay, room_name, (x + 5, y + h - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (120, 120, 120), 1)

    # Draw actions
    for action_name, (x, y, w, h) in actions.items():
        color = (128, 128, 128)  # Grey
        if action_name == current_action:
            color = (0, 255, 255)  # Yellow highlight
        if action_name == previous_action_overlay:
            color = (0, 120, 120)
        cv2.rectangle(overlay, (x, y), (x + w, y + h), color, -1)  # Filled
        cv2.rectangle(overlay, (x, y), (x + w, y + h), (0, 0, 0), 1, lineType=cv2.LINE_AA)


    frame_h, frame_w, _ = frame.shape
    overlay_h, overlay_w, _ = overlay.shape
    x_offset = frame_w - overlay_w - 1300  # 10 pixels from right edge
    y_offset = frame_h - overlay_h - 10  # 10 pixels from bottom edge

    # Ensure overlay fits within the frame
    if overlay_h > frame_h or overlay_w > frame_w:
        raise ValueError("Overlay image is larger than the frame. Resize required.")

    # Create a mask for blending
    overlay_mask = cv2.cvtColor(overlay, cv2.COLOR_BGR2GRAY)
    _, mask = cv2.threshold(overlay_mask, 1, 255, cv2.THRESH_BINARY)

    # Make overlay transparent where needed
    overlay_inv = cv2.bitwise_not(mask)
    roi = frame[y_offset:y_offset+overlay_h, x_offset:x_offset+overlay_w]
    bg = cv2.bitwise_and(roi, roi, mask=overlay_inv)
    fg = cv2.bitwise_and(overlay, overlay, mask=mask)
    frame[y_offset:y_offset+overlay_h, x_offset:x_offset+overlay_w] = cv2.add(bg, fg)



    return frame






In [5]:

#Change to all your local variables
video_path = r"Arveen_3.mp4"
#output_path = r"Arveen_2_Ann.mp4"
excel_file = r"Arveen_3_sheet.xlsx"
start_time_input = "02:32:23"

In [6]:
#2. Run processing and ML
import cv2
import time
from ultralytics import YOLO
from datetime import datetime, timedelta

#xlsx implementation:
import pandas as pd
import openpyxl
from openpyxl.styles import PatternFill



model = YOLO("yolo11n.pt")
model.overrides['verbose'] = False



print(video_path)
#print(output_path)
print(excel_file)

cap = cv2.VideoCapture(video_path)

fps = int(cap.get(cv2.CAP_PROP_FPS))
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
#width = int(600)
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
#height = int(400)

fourcc = cv2.VideoWriter_fourcc(*'mp4v')
#out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

#rooms
test_boxes = [
    (660, 700, 1220, 350), #living room
    (0, 20, 1300, 300), #Kitchen
    (480, 380, 1430, 300), #Bedroom
    (0, 380, 470, 300) #Bathroom
    #(1450, 20, 460, 300) #stairs
]

room_colors = [
    (255, 0, 0),
    (0, 255, 0),
    (0, 0, 255),
    (255, 255, 0)

]

room_names = ["Living Room", "Kitchen", "Bedroom", "Bathroom"] ## add stairs

action_boxes = [
    (1568, 540, 250, 130), #sitting / laying on bed
    (673, 770, 200, 180), #sitting on couch
    (375, 45, 100, 200), # Fridge
    
    (170, 388, 180, 250), # sink
    (1050, 450, 70, 140),#open drawer
    (1500, 370, 140, 120)#use Dresser


    #(120, 40, 160, 120),#cupboard
    #(730, 120, 160, 140)#sit at kitchen table

    #(0, 388, 150, 250)   #Shower
]

action_colors = [
    (255, 0, 0), #sitting / laying on bed
    (0, 0, 255), #sitting on couch
    (0, 0, 255),  #fridge
    (0, 0, 255),  #sink
    (0, 0, 255), #drawer
    (0, 0, 255) #dresser

    #(0, 0, 255), #cupboard
    #(0, 0, 255) #table

    #(0, 255, 255)   #shower
]

action_names = ["Laying on Bed", "Sitting on Couch", "Fridge", "Sink", "Drawer", 
                "Dresser"]#, "Cupboard", "Table"]
previous_action_overlay = None
current_room, current_action = None, None

previous_room = None
previous_action = None

action_start = 0
activity_list = []

#buffers
buffer_frames = 0
room_buffer = 0
room_buffer_threshold = 10
buffer_frames_threshold = 60

total_inf_time = 0.0
start_time = datetime.strptime(start_time_input, "%H:%M:%S")

frame_count = 0





while cap.isOpened():

    success, frame = cap.read()

    if success:

        #time inference

        start_inf = time.perf_counter()
        results = model.track(frame, persist=True)
        end_inf = time.perf_counter()

        total_inf_time += end_inf - start_inf


        elapsed_seconds = frame_count / fps
        current_time = start_time + timedelta(seconds=elapsed_seconds)

        detected_room = None

        #for every "room"
        for idx, room_box in enumerate(test_boxes):
            person_in_room = False
            room_x, room_y, room_w, room_h = room_box
            room_color = room_colors[idx]
            room_name = room_names[idx]

            #check person box
            for box in results[0].boxes:
                if box.cls == 0:
                    x1, y1, x2, y2 = map(int, box.xyxy[0].tolist())

                    #cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 255), 2)
                    #intersection
                    if (
                        x1 < room_x + room_w and x2 > room_x and
                        y1 < room_y + room_h and y2 > room_y
                    ):
                        person_in_room = True
                        detected_room = idx;
                        break

            ## draw room
            #cv2.rectangle(frame, (room_x, room_y), (room_x + room_w, room_y + room_h), room_color, 2)

            #if person_in_room:
                #cv2.putText(frame, f"Person in {room_name}", (room_x + 10, room_y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)


        #if there is no room present
            #incriment and check buffer frames,
            #if exceeded buffer frames

        if detected_room is not None:  # Room detected
            room_buffer += 1
            if detected_room != previous_room:  #different room detected
                if previous_room is not None:  # leave room and print previous
                    activity_list.append(f"Person Left {room_names[previous_room]} at {current_time.strftime('%H:%M:%S')}")
                    print(f"Person Left {room_names[previous_room]} at {current_time.strftime('%H:%M:%S')}")
                    previous_room = None

                # New Room
                if(room_buffer > room_buffer_threshold):
                    activity_list.append(f"Person Entered {room_names[detected_room]} at {current_time.strftime('%H:%M:%S')}")
                    print(f"Person Entered {room_names[detected_room]} at {current_time.strftime('%H:%M:%S')}")
                    current_room = room_names[detected_room]
                    previous_room = detected_room
                    room_buffer = 0

            # draw room
            #cv2.rectangle(frame, (room_x, room_y), (room_x + room_w, room_y + room_h), room_color, 2)
            #if person_in_room:
                #cv2.putText(frame, f"Person in {room_name}", (room_x + 10, room_y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)


        #ACTION CHECK
        action_idx = None
        for idx, action_box in enumerate(action_boxes):
              action_occured = False
              room_x, room_y, room_w, room_h = action_box
              action_color = action_colors[idx]
              action_name = action_names[idx]


              #check every person box
              for box in results[0].boxes:
                  if box.cls == 0:
                      x1, y1, x2, y2 = map(int, box.xyxy[0].tolist())

                      #cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 255), 2)
                      #intersection
                      if (
                          x1 < room_x + room_w and x2 > room_x and
                          y1 < room_y + room_h and y2 > room_y
                      ):
                          action_occured = True
                          action_idx = idx
                          break



        #this code is buggy, repeated entries or none at all at times, something to do with buffer_frames and not resetting
        if action_idx is not None:  # action detected
            buffer_frames = 0
            if action_idx != previous_action:  # New action
                activity_list.append(f"Person Started {action_names[action_idx]} at {current_time.strftime('%H:%M:%S')}")
                print(f"Person Started {action_names[action_idx]} at {current_time.strftime('%H:%M:%S')}")
                current_action = action_names[action_idx]
                
                previous_action = action_idx
                action_start = 1
                # draw action
                #cv2.rectangle(frame, (room_x, room_y), (room_x + room_w, room_y + room_h), action_color, 2)
                # If an action, display text
                #if action_occured:
                    #cv2.putText(frame, f"Person is {action_names[action_idx]}", (room_x + 10, room_y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
                buffer_frames = 0

        elif(previous_action is not None and action_start):
            buffer_frames += 1
            if (buffer_frames > buffer_frames_threshold):

                activity_list.append(f"Person Stopped {action_names[previous_action]} at {current_time.strftime('%H:%M:%S')}")
                print(f"Person Stopped {action_names[previous_action]} at {current_time.strftime('%H:%M:%S')}")
                previous_action_overlay = action_names[previous_action] #overlay logic
                current_action = None #
                buffer_frames = 0
                action_start = 0
                #previous_action = action_idx


        frame = draw_overlay(frame, current_room, current_action, previous_action_overlay)
        #draw_frame(frame)
        #out.write(frame)
        
        frame_count += 1

    else:

        break


cap.release()
#out.release()

average_inference_time = total_inf_time / frame_count
fps = 1 / average_inference_time
print()
print(f"Average Inference Time: {average_inference_time}")
print(f"Frames per second: {fps:.2f}")

#print(f"Output Video Path: {output_path}")


#Excel Sheet
wb = openpyxl.Workbook()
sheet = wb.active
sheet.title = "Activity Log"


room_colors = {
  # "Stairs": "FFCCCC",       #Red
    "Bedroom": "CCE5FF",      #Blue
    "Living Room": "D5E8D4",  #Green
    "Kitchen": "FFD6CC",      #Yellow
    "Bathroom": "FFF2CC"      #Orange
}

#header
sheet.append(["Activity", "Time"])


for entry in activity_list:

    parts = entry.split(' ')
    action = ' '.join(parts[1:3])  #modify later
    room = parts[2]
    if room == "Living":
      room += (" Room")
    timestamp = parts[-1]

    #new row
    row_index = sheet.max_row + 1
    sheet.append([action, timestamp])

    #Color each room
    if room in room_colors:
        color_fill = PatternFill(start_color=room_colors[room], end_color=room_colors[room], fill_type="solid")
        sheet.cell(row=row_index, column=1).fill = color_fill


wb.save(excel_file)
print(f"Exported to Path: {excel_file}")

pd.set_option('display.max_rows', None)
df = pd.read_excel(excel_file)
df




Arveen_3.mp4
Arveen_3_sheet.xlsx
Person Entered Living Room at 02:33:25
Person Left Living Room at 02:33:36
Person Entered Bathroom at 02:33:36
Person Started Sink at 02:33:36
Person Stopped Sink at 02:33:46
Person Left Bathroom at 02:33:47
Person Entered Kitchen at 02:33:47
Person Started Fridge at 02:33:48
Person Stopped Fridge at 02:33:51
Person Left Kitchen at 02:34:02
Person Entered Bathroom at 02:34:02
Person Started Sink at 02:34:02
Person Stopped Sink at 02:34:18
Person Left Bathroom at 02:37:31
Person Entered Living Room at 02:37:31
Person Left Living Room at 02:39:14
Person Entered Bedroom at 02:39:14
Person Started Drawer at 02:39:15
Person Stopped Drawer at 02:39:17
Person Left Bedroom at 02:39:22
Person Entered Living Room at 02:39:22
Person Left Living Room at 02:39:50
Person Entered Bathroom at 02:39:50
Person Started Sink at 02:39:50
Person Stopped Sink at 02:39:54
Person Left Bathroom at 02:42:28
Person Entered Kitchen at 02:42:28
Person Started Fridge at 02:42:28
Pers

,Activity,Time
0,Entered Living,02:33:25
1,Left Living,02:33:36
2,Entered Bathroom,02:33:36
3,Started Sink,02:33:36
4,Stopped Sink,02:33:46
5,Left Bathroom,02:33:47
6,Entered Kitchen,02:33:47
7,Started Fridge,02:33:48
8,Stopped Fridge,02:33:51
9,Left Kitchen,02:34:02


In [7]:

#Change to all your local variables
video_path = r"Arveen_4.mp4"
#output_path = r"Arveen_2_Ann.mp4"
excel_file = r"Arveen_4_sheet.xlsx"
start_time_input = "03:01:48"

In [8]:
#2. Run processing and ML
import cv2
import time
from ultralytics import YOLO
from datetime import datetime, timedelta

#xlsx implementation:
import pandas as pd
import openpyxl
from openpyxl.styles import PatternFill



model = YOLO("yolo11n.pt")
model.overrides['verbose'] = False



print(video_path)
#print(output_path)
print(excel_file)

cap = cv2.VideoCapture(video_path)

fps = int(cap.get(cv2.CAP_PROP_FPS))
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
#width = int(600)
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
#height = int(400)

fourcc = cv2.VideoWriter_fourcc(*'mp4v')
#out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

#rooms
test_boxes = [
    (660, 700, 1220, 350), #living room
    (0, 20, 1300, 300), #Kitchen
    (480, 380, 1430, 300), #Bedroom
    (0, 380, 470, 300) #Bathroom
    #(1450, 20, 460, 300) #stairs
]

room_colors = [
    (255, 0, 0),
    (0, 255, 0),
    (0, 0, 255),
    (255, 255, 0)

]

room_names = ["Living Room", "Kitchen", "Bedroom", "Bathroom"] ## add stairs

action_boxes = [
    (1568, 540, 250, 130), #sitting / laying on bed
    (673, 770, 200, 180), #sitting on couch
    (375, 45, 100, 200), # Fridge
    
    (170, 388, 180, 250), # sink
    (1050, 450, 70, 140),#open drawer
    (1500, 370, 140, 120)#use Dresser


    #(120, 40, 160, 120),#cupboard
    #(730, 120, 160, 140)#sit at kitchen table

    #(0, 388, 150, 250)   #Shower
]

action_colors = [
    (255, 0, 0), #sitting / laying on bed
    (0, 0, 255), #sitting on couch
    (0, 0, 255),  #fridge
    (0, 0, 255),  #sink
    (0, 0, 255), #drawer
    (0, 0, 255) #dresser

    #(0, 0, 255), #cupboard
    #(0, 0, 255) #table

    #(0, 255, 255)   #shower
]

action_names = ["Laying on Bed", "Sitting on Couch", "Fridge", "Sink", "Drawer", 
                "Dresser"]#, "Cupboard", "Table"]
previous_action_overlay = None
current_room, current_action = None, None

previous_room = None
previous_action = None

action_start = 0
activity_list = []

#buffers
buffer_frames = 0
room_buffer = 0
room_buffer_threshold = 10
buffer_frames_threshold = 60

total_inf_time = 0.0
start_time = datetime.strptime(start_time_input, "%H:%M:%S")

frame_count = 0





while cap.isOpened():

    success, frame = cap.read()

    if success:

        #time inference

        start_inf = time.perf_counter()
        results = model.track(frame, persist=True)
        end_inf = time.perf_counter()

        total_inf_time += end_inf - start_inf


        elapsed_seconds = frame_count / fps
        current_time = start_time + timedelta(seconds=elapsed_seconds)

        detected_room = None

        #for every "room"
        for idx, room_box in enumerate(test_boxes):
            person_in_room = False
            room_x, room_y, room_w, room_h = room_box
            room_color = room_colors[idx]
            room_name = room_names[idx]

            #check person box
            for box in results[0].boxes:
                if box.cls == 0:
                    x1, y1, x2, y2 = map(int, box.xyxy[0].tolist())

                    #cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 255), 2)
                    #intersection
                    if (
                        x1 < room_x + room_w and x2 > room_x and
                        y1 < room_y + room_h and y2 > room_y
                    ):
                        person_in_room = True
                        detected_room = idx;
                        break

            ## draw room
            #cv2.rectangle(frame, (room_x, room_y), (room_x + room_w, room_y + room_h), room_color, 2)

            #if person_in_room:
                #cv2.putText(frame, f"Person in {room_name}", (room_x + 10, room_y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)


        #if there is no room present
            #incriment and check buffer frames,
            #if exceeded buffer frames

        if detected_room is not None:  # Room detected
            room_buffer += 1
            if detected_room != previous_room:  #different room detected
                if previous_room is not None:  # leave room and print previous
                    activity_list.append(f"Person Left {room_names[previous_room]} at {current_time.strftime('%H:%M:%S')}")
                    print(f"Person Left {room_names[previous_room]} at {current_time.strftime('%H:%M:%S')}")
                    previous_room = None

                # New Room
                if(room_buffer > room_buffer_threshold):
                    activity_list.append(f"Person Entered {room_names[detected_room]} at {current_time.strftime('%H:%M:%S')}")
                    print(f"Person Entered {room_names[detected_room]} at {current_time.strftime('%H:%M:%S')}")
                    current_room = room_names[detected_room]
                    previous_room = detected_room
                    room_buffer = 0

            # draw room
            #cv2.rectangle(frame, (room_x, room_y), (room_x + room_w, room_y + room_h), room_color, 2)
            #if person_in_room:
                #cv2.putText(frame, f"Person in {room_name}", (room_x + 10, room_y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)


        #ACTION CHECK
        action_idx = None
        for idx, action_box in enumerate(action_boxes):
              action_occured = False
              room_x, room_y, room_w, room_h = action_box
              action_color = action_colors[idx]
              action_name = action_names[idx]


              #check every person box
              for box in results[0].boxes:
                  if box.cls == 0:
                      x1, y1, x2, y2 = map(int, box.xyxy[0].tolist())

                      #cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 255), 2)
                      #intersection
                      if (
                          x1 < room_x + room_w and x2 > room_x and
                          y1 < room_y + room_h and y2 > room_y
                      ):
                          action_occured = True
                          action_idx = idx
                          break



        #this code is buggy, repeated entries or none at all at times, something to do with buffer_frames and not resetting
        if action_idx is not None:  # action detected
            buffer_frames = 0
            if action_idx != previous_action:  # New action
                activity_list.append(f"Person Started {action_names[action_idx]} at {current_time.strftime('%H:%M:%S')}")
                print(f"Person Started {action_names[action_idx]} at {current_time.strftime('%H:%M:%S')}")
                current_action = action_names[action_idx]
                
                previous_action = action_idx
                action_start = 1
                # draw action
                #cv2.rectangle(frame, (room_x, room_y), (room_x + room_w, room_y + room_h), action_color, 2)
                # If an action, display text
                #if action_occured:
                    #cv2.putText(frame, f"Person is {action_names[action_idx]}", (room_x + 10, room_y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
                buffer_frames = 0

        elif(previous_action is not None and action_start):
            buffer_frames += 1
            if (buffer_frames > buffer_frames_threshold):

                activity_list.append(f"Person Stopped {action_names[previous_action]} at {current_time.strftime('%H:%M:%S')}")
                print(f"Person Stopped {action_names[previous_action]} at {current_time.strftime('%H:%M:%S')}")
                previous_action_overlay = action_names[previous_action] #overlay logic
                current_action = None #
                buffer_frames = 0
                action_start = 0
                #previous_action = action_idx


        frame = draw_overlay(frame, current_room, current_action, previous_action_overlay)
        #draw_frame(frame)
        #out.write(frame)
        
        frame_count += 1

    else:

        break


cap.release()
#out.release()

average_inference_time = total_inf_time / frame_count
fps = 1 / average_inference_time
print()
print(f"Average Inference Time: {average_inference_time}")
print(f"Frames per second: {fps:.2f}")

#print(f"Output Video Path: {output_path}")


#Excel Sheet
wb = openpyxl.Workbook()
sheet = wb.active
sheet.title = "Activity Log"


room_colors = {
  # "Stairs": "FFCCCC",       #Red
    "Bedroom": "CCE5FF",      #Blue
    "Living Room": "D5E8D4",  #Green
    "Kitchen": "FFD6CC",      #Yellow
    "Bathroom": "FFF2CC"      #Orange
}

#header
sheet.append(["Activity", "Time"])


for entry in activity_list:

    parts = entry.split(' ')
    action = ' '.join(parts[1:3])  #modify later
    room = parts[2]
    if room == "Living":
      room += (" Room")
    timestamp = parts[-1]

    #new row
    row_index = sheet.max_row + 1
    sheet.append([action, timestamp])

    #Color each room
    if room in room_colors:
        color_fill = PatternFill(start_color=room_colors[room], end_color=room_colors[room], fill_type="solid")
        sheet.cell(row=row_index, column=1).fill = color_fill


wb.save(excel_file)
print(f"Exported to Path: {excel_file}")

pd.set_option('display.max_rows', None)
df = pd.read_excel(excel_file)
df




Arveen_4.mp4
Arveen_4_sheet.xlsx
Person Entered Living Room at 03:04:05
Person Left Living Room at 03:04:12
Person Entered Bedroom at 03:04:12
Person Started Laying on Bed at 03:04:12
Person Stopped Laying on Bed at 03:04:15
Person Left Bedroom at 03:04:27
Person Entered Bathroom at 03:04:27
Person Started Sink at 03:04:27
Person Stopped Sink at 03:04:34
Person Left Bathroom at 03:08:30
Person Entered Living Room at 03:08:30
Person Left Living Room at 03:10:21
Person Entered Bathroom at 03:10:21
Person Left Bathroom at 03:14:57
Person Entered Living Room at 03:14:57
Person Started Sitting on Couch at 03:15:47
Person Stopped Sitting on Couch at 03:15:50
Person Left Living Room at 03:16:11
Person Entered Bathroom at 03:16:11
Person Started Sink at 03:16:11
Person Stopped Sink at 03:16:15
Person Left Bathroom at 03:21:07
Person Entered Kitchen at 03:21:07
Person Started Fridge at 03:21:07
Person Stopped Fridge at 03:21:10
Person Left Kitchen at 03:23:05
Person Entered Living Room at 03:23

,Activity,Time
0,Entered Living,03:04:05
1,Left Living,03:04:12
2,Entered Bedroom,03:04:12
3,Started Laying,03:04:12
4,Stopped Laying,03:04:15
5,Left Bedroom,03:04:27
6,Entered Bathroom,03:04:27
7,Started Sink,03:04:27
8,Stopped Sink,03:04:34
9,Left Bathroom,03:08:30


In [9]:

#Change to all your local variables
video_path = r"Arveen_5.mp4"
#output_path = r"Arveen_2_Ann.mp4"
excel_file = r"Arveen_5_sheet.xlsx"
start_time_input = "03:37:40"

In [10]:
#2. Run processing and ML
import cv2
import time
from ultralytics import YOLO
from datetime import datetime, timedelta

#xlsx implementation:
import pandas as pd
import openpyxl
from openpyxl.styles import PatternFill



model = YOLO("yolo11n.pt")
model.overrides['verbose'] = False



print(video_path)
#print(output_path)
print(excel_file)

cap = cv2.VideoCapture(video_path)

fps = int(cap.get(cv2.CAP_PROP_FPS))
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
#width = int(600)
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
#height = int(400)

fourcc = cv2.VideoWriter_fourcc(*'mp4v')
#out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

#rooms
test_boxes = [
    (660, 700, 1220, 350), #living room
    (0, 20, 1300, 300), #Kitchen
    (480, 380, 1430, 300), #Bedroom
    (0, 380, 470, 300) #Bathroom
    #(1450, 20, 460, 300) #stairs
]

room_colors = [
    (255, 0, 0),
    (0, 255, 0),
    (0, 0, 255),
    (255, 255, 0)

]

room_names = ["Living Room", "Kitchen", "Bedroom", "Bathroom"] ## add stairs

action_boxes = [
    (1568, 540, 250, 130), #sitting / laying on bed
    (673, 770, 200, 180), #sitting on couch
    (375, 45, 100, 200), # Fridge
    
    (170, 388, 180, 250), # sink
    (1050, 450, 70, 140),#open drawer
    (1500, 370, 140, 120)#use Dresser


    #(120, 40, 160, 120),#cupboard
    #(730, 120, 160, 140)#sit at kitchen table

    #(0, 388, 150, 250)   #Shower
]

action_colors = [
    (255, 0, 0), #sitting / laying on bed
    (0, 0, 255), #sitting on couch
    (0, 0, 255),  #fridge
    (0, 0, 255),  #sink
    (0, 0, 255), #drawer
    (0, 0, 255) #dresser

    #(0, 0, 255), #cupboard
    #(0, 0, 255) #table

    #(0, 255, 255)   #shower
]

action_names = ["Laying on Bed", "Sitting on Couch", "Fridge", "Sink", "Drawer", 
                "Dresser"]#, "Cupboard", "Table"]
previous_action_overlay = None
current_room, current_action = None, None

previous_room = None
previous_action = None

action_start = 0
activity_list = []

#buffers
buffer_frames = 0
room_buffer = 0
room_buffer_threshold = 10
buffer_frames_threshold = 60

total_inf_time = 0.0
start_time = datetime.strptime(start_time_input, "%H:%M:%S")

frame_count = 0





while cap.isOpened():

    success, frame = cap.read()

    if success:

        #time inference

        start_inf = time.perf_counter()
        results = model.track(frame, persist=True)
        end_inf = time.perf_counter()

        total_inf_time += end_inf - start_inf


        elapsed_seconds = frame_count / fps
        current_time = start_time + timedelta(seconds=elapsed_seconds)

        detected_room = None

        #for every "room"
        for idx, room_box in enumerate(test_boxes):
            person_in_room = False
            room_x, room_y, room_w, room_h = room_box
            room_color = room_colors[idx]
            room_name = room_names[idx]

            #check person box
            for box in results[0].boxes:
                if box.cls == 0:
                    x1, y1, x2, y2 = map(int, box.xyxy[0].tolist())

                    #cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 255), 2)
                    #intersection
                    if (
                        x1 < room_x + room_w and x2 > room_x and
                        y1 < room_y + room_h and y2 > room_y
                    ):
                        person_in_room = True
                        detected_room = idx;
                        break

            ## draw room
            #cv2.rectangle(frame, (room_x, room_y), (room_x + room_w, room_y + room_h), room_color, 2)

            #if person_in_room:
                #cv2.putText(frame, f"Person in {room_name}", (room_x + 10, room_y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)


        #if there is no room present
            #incriment and check buffer frames,
            #if exceeded buffer frames

        if detected_room is not None:  # Room detected
            room_buffer += 1
            if detected_room != previous_room:  #different room detected
                if previous_room is not None:  # leave room and print previous
                    activity_list.append(f"Person Left {room_names[previous_room]} at {current_time.strftime('%H:%M:%S')}")
                    print(f"Person Left {room_names[previous_room]} at {current_time.strftime('%H:%M:%S')}")
                    previous_room = None

                # New Room
                if(room_buffer > room_buffer_threshold):
                    activity_list.append(f"Person Entered {room_names[detected_room]} at {current_time.strftime('%H:%M:%S')}")
                    print(f"Person Entered {room_names[detected_room]} at {current_time.strftime('%H:%M:%S')}")
                    current_room = room_names[detected_room]
                    previous_room = detected_room
                    room_buffer = 0

            # draw room
            #cv2.rectangle(frame, (room_x, room_y), (room_x + room_w, room_y + room_h), room_color, 2)
            #if person_in_room:
                #cv2.putText(frame, f"Person in {room_name}", (room_x + 10, room_y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)


        #ACTION CHECK
        action_idx = None
        for idx, action_box in enumerate(action_boxes):
              action_occured = False
              room_x, room_y, room_w, room_h = action_box
              action_color = action_colors[idx]
              action_name = action_names[idx]


              #check every person box
              for box in results[0].boxes:
                  if box.cls == 0:
                      x1, y1, x2, y2 = map(int, box.xyxy[0].tolist())

                      #cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 255), 2)
                      #intersection
                      if (
                          x1 < room_x + room_w and x2 > room_x and
                          y1 < room_y + room_h and y2 > room_y
                      ):
                          action_occured = True
                          action_idx = idx
                          break



        #this code is buggy, repeated entries or none at all at times, something to do with buffer_frames and not resetting
        if action_idx is not None:  # action detected
            buffer_frames = 0
            if action_idx != previous_action:  # New action
                activity_list.append(f"Person Started {action_names[action_idx]} at {current_time.strftime('%H:%M:%S')}")
                print(f"Person Started {action_names[action_idx]} at {current_time.strftime('%H:%M:%S')}")
                current_action = action_names[action_idx]
                
                previous_action = action_idx
                action_start = 1
                # draw action
                #cv2.rectangle(frame, (room_x, room_y), (room_x + room_w, room_y + room_h), action_color, 2)
                # If an action, display text
                #if action_occured:
                    #cv2.putText(frame, f"Person is {action_names[action_idx]}", (room_x + 10, room_y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
                buffer_frames = 0

        elif(previous_action is not None and action_start):
            buffer_frames += 1
            if (buffer_frames > buffer_frames_threshold):

                activity_list.append(f"Person Stopped {action_names[previous_action]} at {current_time.strftime('%H:%M:%S')}")
                print(f"Person Stopped {action_names[previous_action]} at {current_time.strftime('%H:%M:%S')}")
                previous_action_overlay = action_names[previous_action] #overlay logic
                current_action = None #
                buffer_frames = 0
                action_start = 0
                #previous_action = action_idx


        frame = draw_overlay(frame, current_room, current_action, previous_action_overlay)
        #draw_frame(frame)
        #out.write(frame)
        
        frame_count += 1

    else:

        break


cap.release()
#out.release()

average_inference_time = total_inf_time / frame_count
fps = 1 / average_inference_time
print()
print(f"Average Inference Time: {average_inference_time}")
print(f"Frames per second: {fps:.2f}")

#print(f"Output Video Path: {output_path}")


#Excel Sheet
wb = openpyxl.Workbook()
sheet = wb.active
sheet.title = "Activity Log"


room_colors = {
  # "Stairs": "FFCCCC",       #Red
    "Bedroom": "CCE5FF",      #Blue
    "Living Room": "D5E8D4",  #Green
    "Kitchen": "FFD6CC",      #Yellow
    "Bathroom": "FFF2CC"      #Orange
}

#header
sheet.append(["Activity", "Time"])


for entry in activity_list:

    parts = entry.split(' ')
    action = ' '.join(parts[1:3])  #modify later
    room = parts[2]
    if room == "Living":
      room += (" Room")
    timestamp = parts[-1]

    #new row
    row_index = sheet.max_row + 1
    sheet.append([action, timestamp])

    #Color each room
    if room in room_colors:
        color_fill = PatternFill(start_color=room_colors[room], end_color=room_colors[room], fill_type="solid")
        sheet.cell(row=row_index, column=1).fill = color_fill


wb.save(excel_file)
print(f"Exported to Path: {excel_file}")

pd.set_option('display.max_rows', None)
df = pd.read_excel(excel_file)
df




Arveen_5.mp4
Arveen_5_sheet.xlsx
Person Entered Living Room at 03:39:00
Person Left Living Room at 03:39:24
Person Entered Bathroom at 03:39:24
Person Started Sink at 03:39:24
Person Stopped Sink at 03:39:27
Person Left Bathroom at 03:43:06
Person Entered Kitchen at 03:43:06
Person Started Fridge at 03:43:07
Person Stopped Fridge at 03:43:10
Person Left Kitchen at 03:44:28
Person Entered Living Room at 03:44:28
Person Left Living Room at 03:44:50
Person Entered Bathroom at 03:44:50
Person Started Sink at 03:44:50
Person Stopped Sink at 03:45:04
Person Left Bathroom at 03:47:35
Person Entered Kitchen at 03:47:35
Person Started Fridge at 03:47:36
Person Stopped Fridge at 03:47:39
Person Left Kitchen at 03:49:27
Person Entered Living Room at 03:49:27
Person Left Living Room at 03:49:39
Person Entered Kitchen at 03:49:39
Person Left Kitchen at 03:49:59
Person Entered Bathroom at 03:49:59
Person Started Sink at 03:49:59
Person Stopped Sink at 03:50:07
Person Left Bathroom at 03:55:19
Person

,Activity,Time
0,Entered Living,03:39:00
1,Left Living,03:39:24
2,Entered Bathroom,03:39:24
3,Started Sink,03:39:24
4,Stopped Sink,03:39:27
5,Left Bathroom,03:43:06
6,Entered Kitchen,03:43:06
7,Started Fridge,03:43:07
8,Stopped Fridge,03:43:10
9,Left Kitchen,03:44:28
